## GPT-2 Small on GSMK8 

The purpose of this is to measure the coherence of an LLM on the Cohence of solving math problems within GSMK8

# Model set up

In [ ]:
# import the hugging face transformers library
from transformers import pipelineß
from transformers import GPT2Tokenizer



#import the gpt2 model
GPT2Tokenizer.from_pretrained('gpt2')
generator = pipeline('text-generation', model='gpt2')


In [ ]:
#The End